# Chatbot for E-commerce

## 1. Data Analysis

In [71]:
import json
import nltk

words=[]
classes = []
documents = []

ignore_words = ['?', '!', ".", ",", "'s"]

data_file = open('intents.json').read()
intents = json.loads(data_file)

In [72]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Nice to meet you.',
    'Hello, Good to see you.',
    'Hi there, how can I help you?.'],
   'context': ['']},
  {'tag': 'introduce',
   'patterns': ['What are you?',
    'Who are you?',
    'Can you introduce yourself?',
    "What's your name?"],
   'responses': ['My name is EMO. I am your Online Shopping Assistant.'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you! Thanks for shopping with us.',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'con

In [73]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [74]:
print("Classes: ", classes, "\n")
print("Words: ", words, "\n")
print("Documents: ", documents, "\n")

Classes:  ['greeting', 'introduce', 'goodbye', 'thanks', 'buybooks', 'payment', 'noanswer'] 

Words:  ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'What', 'are', 'you', '?', 'Who', 'are', 'you', '?', 'Can', 'you', 'introduce', 'yourself', '?', 'What', "'s", 'your', 'name', '?', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'I', 'want', 'to', 'buy', 'a', 'romantic', 'comedy', 'genre', 'book', 'I', 'want', 'to', 'buy', 'the', 'newest', 'novel', 'you', 'have', 'Do', 'you', 'sell', 'books', '?', 'Can', 'you', 'recommend', 'me', 'some', 'books', '?', 'How', 'can', 'i', 'pay', '?', 'I', 'want', 'to', 'pay', 'for', 'this', 'How', 'much', 'money', 'does', 'this', 'cost', '?', 'Check', 'out', 'please', 'I', 'am', 'finished', 'shopping', '.'] 

Documents:  [(['Hi'

## 2. Preprocess data


In [75]:
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
import random
import numpy as np

lemmatizer = WordNetLemmatizer()

In [76]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print(words)

['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [77]:
# sort classes
classes = sorted(list(set(classes)))
classes

['buybooks',
 'goodbye',
 'greeting',
 'introduce',
 'noanswer',
 'payment',
 'thanks']

In [78]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


65 unique lemmatized words ['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [79]:
with open("words_data.json", "w") as words_data:
    json.dump(words, words_data)

In [80]:
with open("classes_data.json", "w") as classes_data:
    json.dump(classes, classes_data)

## 3. Create training and testing data

In [81]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [82]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [83]:
for t in training:
    print(t)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [84]:
output_empty

[0, 0, 0, 0, 0, 0, 0]

In [85]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])
print("Training data created")

Training data created


In [86]:
y_train

[[0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0]]

In [87]:
for i in range(len(X_train)):
    print(X_train[i], y_train[i])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] [0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## 4.1 Build the Sequential Model

In [88]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils import plot_model

In [89]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [90]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [91]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               8448      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 455       
Total params: 17,159
Trainable params: 17,159
Non-trainable params: 0
_________________________________________________________________


In [92]:
#fitting and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
7/7 [==============================] - 0s 9ms/step - loss: 1.9894 - accuracy: 0.1613
Epoch 2/200
7/7 [==============================] - 0s 3ms/step - loss: 1.9392 - accuracy: 0.2258
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 1.7673 - accuracy: 0.3548
Epoch 4/200
7/7 [==============================] - 0s 3ms/step - loss: 1.7357 - accuracy: 0.4194
Epoch 5/200
7/7 [==============================] - 0s 3ms/step - loss: 1.6722 - accuracy: 0.3871
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5998 - accuracy: 0.5161
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.6409 - accuracy: 0.4516
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5264 - accuracy: 0.4194
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4321 - accuracy: 0.4194
Epoch 10/200
7/7 [==============================] - 0s 3ms/step - loss: 1.2860 - accuracy: 0.6774
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 2ms/step - loss: 0.1765 - accuracy: 0.9355
Epoch 85/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0545 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0745 - accuracy: 1.0000
Epoch 87/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1150 - accuracy: 0.9677
Epoch 88/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0702 - accuracy: 1.0000
Epoch 89/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1198 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0855 - accuracy: 1.0000
Epoch 91/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0544 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0621 - accuracy: 1.0000
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0313 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0510 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0474 - accuracy: 1.0000
Epoch 176/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 177/200
7/7 [=====

In [93]:
# plot_model(model, to_file = "model.png", show_shapes=True)